# **Introduction**

_Students write text here_

# **1. Basic Preprocessing**

_Students write text here_

## **1.1. Load Data**

In [ ]:
# Matteo writes code here

## **1.2. Interpolate the Pulse Artefact: Rationale & Execution**

In [ ]:
# Matteo writes code here

## **1.3. High-Pass Filtering: Rationale & Execution**

_Students write text here_

In [ ]:
# Matteo writes code here

## **1.4. Downsampling: Rationale & Execution**

In [ ]:
# Matteo writes code here

# **2. Independent Component Analysis (ICA)**

## **2.1. Rationale**

_Students write text here_

## **2.2. Fitting**

In [ ]:
# Matteo writes code here

## **2.3. Components Selection**

In [ ]:
# Matteo writes code here

# **3. Manual Artifact Rejection**

## **3.1. Rationale**

_Students write text here_

## **3.2. Execution**

In [ ]:
# Matteo writes code here

# **4. Assessing & Computing a Transcranial Evoked Potential (TEP)**

## **4.1. Rationale**

_Students write text here_

## **4.2. Execution**

In [ ]:
# Matteo writes code here